In speech processing we may use really huge networks cause majority of the processing bottleneck is on CPU and running the CTC function...

Also, CLARIN mobile corpus seems to be too small for advanced network

In [1]:
#!/usr/bin/python3

# For demonstration purposes - Paweł Tomasik
# for CLARIN_MOBILE - generally it is unnormalized

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

from keras.models import Model
from keras.layers import LSTM, Conv1D, Dropout, LeakyReLU, Dense, Input, Lambda, TimeDistributed, Flatten, Conv2D, BatchNormalization, GRU, Bidirectional
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.initializers import Orthogonal
from keras.callbacks import Callback

from keras.backend import ctc_batch_cost, expand_dims
import keras.backend as K

import editdistance  # For digit error rate
import keras
import librosa
import numpy as np
import os

import gc
X = Y = None
gc.collect()

LENGTH = 1700 * 128
RECS = 1384 * 4
TRANSL = 307

X = np.zeros([RECS, LENGTH, 1], np.float32)
Y = np.zeros([RECS, TRANSL], np.int16)
counter = 0

for i in range(4):
    Xpart = np.load("datasets/clarin-long/data/clarin-mfcc-rec-pure-timedomain-{}.npy".format(i))
    Ypart = np.load("datasets/clarin-long/data/clarin-mfcc-trans-pure-timedomain-{}.npy".format(i))
    recs = Xpart.shape[0]
    reclen = Xpart.shape[1]
    translen = Ypart.shape[1]
    X[counter : counter + recs, :reclen, :] = Xpart
    Y[counter : counter + recs, :translen] = Ypart
    counter += recs
    
print(counter, RECS)
counter //= 32
counter *= 32

X = X[:counter]
Y = Y[:counter]

Xpart.shape

#X = np.clip(X, -3, 3)
MEAN = X.mean()
STD = X.std()

valid = np.random.random(X.shape[0]) > 0.9

/venv/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


5523 5536


In [2]:
import matplotlib.pyplot as plt
first = lambda x: len(x) - np.where(x[::-1].cumsum())[0][0]
first(X[0])

138560

In [3]:
len(lang)

NameError: name 'lang' is not defined

In [4]:
lang = {'dz': "affr",
  'b': "plos",
  'zi': "fric",
  'dZ': "affr",
  'Z': "fric",
  't': "plos",
  'f': "fric",
  'ts': "affr",
  'i': "vowel",
  'I': "vowel",
  'o': "vowel",
  'si': "fric",
  'j': "other",
  's': "fric",
  'g': "plos",
  'on': "vowel",
  'r': "other",
  'ni': "other",
  'sil': "noise",
  'z': "fric",
  'a': "vowel",
  'v': "fric",
  'e': "vowel",
  'd': "plos",
  'm': "other",
  'k': "plos",
  'u': "vowel",
  'S': "fric",
  'w': "other",
  'x': "fric",
  'l': "other",
  'tsi': "affr",
  'n': "other",
  'tS': "affr",
  'dzi': "affr",
  'p': "plos",
  'en': "vowel"}

categories = [x for x in set(lang.values())]
cats = {cat: [(ix, x) for ix, x in enumerate(lang) if lang[x] == cat] for cat in categories}
cats

{'affr': [(2, 'tS'),
  (5, 'tsi'),
  (6, 'dZ'),
  (7, 'dzi'),
  (30, 'ts'),
  (36, 'dz')],
 'fric': [(0, 'v'),
  (4, 'Z'),
  (12, 'S'),
  (16, 'zi'),
  (21, 'z'),
  (23, 's'),
  (26, 'f'),
  (27, 'x'),
  (34, 'si')],
 'noise': [(19, 'sil')],
 'other': [(3, 'm'),
  (8, 'w'),
  (9, 'j'),
  (13, 'r'),
  (14, 'l'),
  (20, 'ni'),
  (29, 'n')],
 'plos': [(1, 'g'), (11, 'p'), (15, 'd'), (28, 'k'), (33, 't'), (35, 'b')],
 'vowel': [(10, 'u'),
  (17, 'e'),
  (18, 'en'),
  (22, 'i'),
  (24, 'on'),
  (25, 'I'),
  (31, 'o'),
  (32, 'a')]}

In [5]:
mapping = {cat: [ix for ix, x in enumerate(lang) if lang[x] == cat] for cat in categories}
mapping = [(x, mapping[x].index(ix)) for ix, x in enumerate(lang.values())]
print(mapping)

def mix_softmax(list_of_layers):
    other, fric, plos, vowel, affr, noise = list_of_layers
    fromstring = {"other": other, "fric": fric, "plos": plos, "vowel": vowel, "affr": affr, "noise": noise}
    return K.stack([fromstring[x[0]][:, :, x[1]] for x in mapping], axis=-1)

[('fric', 0), ('plos', 0), ('affr', 0), ('other', 0), ('fric', 1), ('affr', 1), ('affr', 2), ('affr', 3), ('other', 1), ('other', 2), ('vowel', 0), ('plos', 1), ('fric', 2), ('other', 3), ('other', 4), ('plos', 2), ('fric', 3), ('vowel', 1), ('vowel', 2), ('noise', 0), ('other', 5), ('fric', 4), ('vowel', 3), ('fric', 5), ('vowel', 4), ('vowel', 5), ('fric', 6), ('fric', 7), ('plos', 3), ('other', 6), ('affr', 4), ('vowel', 6), ('vowel', 7), ('plos', 4), ('fric', 8), ('plos', 5), ('affr', 5)]


In [6]:
lens = [len(cats[x]) for x in ["other", "fric", "plos", "vowel", "affr", "noise"]]
lens

[7, 9, 6, 8, 6, 1]

In [7]:
from keras.constraints import max_norm
from tensorflow.spectral import dct
import tensorflow as tf

shift = np.exp(2 * np.pi * 1j / 512)

def spectral_coeffs(x):
    return dct(K.log(x**2 + 2e-12))[:, :, :20]

class CZT(keras.layers.Layer):
    def build(self, input_shape):
        self.z = self.add_weight(shape=[2], initializer=lambda shape: K.variable([np.real(shift), np.imag(shift)]),
                                 name='kernel_z', constraint=max_norm(1.))
        self.w = self.add_weight(shape=[2], initializer=lambda shape: K.variable([1, 0]),
                                 name='kernel_w', constraint=max_norm(1.))
    
    def compute_output_shape(self, input_shape):
        assert input_shape and len(input_shape) >= 2
        assert input_shape[-1]
        output_shape = list(input_shape)
        output_shape[-1] = 257
        return tuple(output_shape)
    
    def call(self, inputs):
        ints = K.reshape(K.arange(257, dtype='complex64'), [257, 1])
        k = K.reshape(K.arange(512, dtype='complex64'), [1, 512])
        z = K.cast(self.z[0], dtype='complex64_ref') + 1j * K.cast(self.z[1], dtype='complex64_ref')
        w = K.cast(self.w[0], dtype='complex64_ref') + 1j * K.cast(self.w[1], dtype='complex64_ref')
        weights = K.dot(z * K.ones([257, 1], dtype='complex64'), K.reshape(w, [1, -1]) ** (-k)) ** (-ints)
        print(z.shape, w.shape, weights.shape)
        czt = K.dot(K.cast(inputs, dtype='complex64_ref'), K.transpose(weights))
        return K.abs(czt)

def mk_model(max_label_length):
    feature_input = Input(shape = (None, 1))
    conv_layer = Conv1D(512, 512, strides=128, activation='relu', bias=False)
    conv_layer.build((None, 1))
    conv_layer.trainable = False
    conv_layer.set_weights([np.eye(512, dtype=np.float32).reshape([512, 1, 512])])
    conv_layer = conv_layer(feature_input)
    layer = CZT()(conv_layer)
    layer = Lambda(spectral_coeffs)(layer)
    layer = Lambda(K.expand_dims)(layer)
    layer_1 = Conv2D(12, [5,1], activation='linear', strides=(2,1), kernel_initializer=Orthogonal(), padding='same')(layer)
    layer_2 = LeakyReLU(0.01)(layer_1)
    layer = BatchNormalization()(layer_2)
    layer_1 = Conv2D(16, [5,1], activation='linear', strides=(2,1), kernel_initializer=Orthogonal(), padding='same')(layer)
    layer_2 = LeakyReLU(0.01)(layer_1)
    layer = BatchNormalization()(layer_2)
    layer = TimeDistributed(Flatten())(layer)
    layer_1 = Conv1D(512, 5, activation='linear', kernel_initializer=Orthogonal(), padding='same')(layer)
    layer_2 = LeakyReLU(0.01)(layer_1)
    intro_to_columns = BatchNormalization()(layer_2)
    column_heads = []
    for act_len, size in zip(lens, [128, 128, 96, 128, 128, 48]):
        layer = intro_to_columns
        for i in range(3):
            layer = Dense(size, kernel_initializer=Orthogonal(), activation='linear')(layer)
            layer = LeakyReLU(0.01)(layer)
            layer = BatchNormalization()(layer)
        for i in range(3):
            layer = GRU(size, return_sequences = True, recurrent_dropout=0.01, kernel_initializer=Orthogonal(), activation='linear')(layer)
            layer = LeakyReLU(0.01)(layer)
            layer = BatchNormalization()(layer)
        layer = Dense(act_len)(layer)
        layer = LeakyReLU(0.01)(layer)
        layer = BatchNormalization()(layer)
        column_heads.append(layer)
    layer_14 = Lambda(mix_softmax)(column_heads)
    layer_15 = Lambda(lambda x: K.softmax(tf.pad(x, [[0, 0], [0, 0], [0, 1]])))(layer_14)
    label_input = Input(shape = (max_label_length,))
    input_length = Input(name='input_length', shape=[1], dtype='int64')
    label_length = Input(name='label_length', shape=[1], dtype='int64')
    loss_lambda = Lambda(lambda args:ctc_batch_cost(*args), output_shape=(1,), name='ctc')([label_input, layer_15, input_length, label_length])
    model = Model([feature_input, label_input, input_length, label_length], [loss_lambda])
    model.summary()
    predictive = Model(feature_input, layer_15)
    return model, predictive

mk_model(None)

/venv/lib/python3.5/site-packages/ipykernel_launcher.py:36: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(512, 512, use_bias=False, activation="relu", strides=128)`


() () (257, 512)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, 1)      0                                            
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, None, 512)    262144      input_1[0][0]                    
__________________________________________________________________________________________________
czt_1 (CZT)                     (None, None, 257)    4           conv1d_1[0][0]                   
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, None, 20)     0           czt_1[0][0]                      
____________________________________________________________________________________________

(<keras.engine.training.Model at 0x7fa8e4fb1320>,
 <keras.engine.training.Model at 0x7fa8e4fce438>)

In [ ]:
mc = keras.callbacks.ModelCheckpoint('models/mfcc-ctc-{epoch:08d}-bigger.h5', 
                                     save_weights_only=False, period=5)

class MyCallback(keras.callbacks.Callback):
    def __init__(self, fname):
        import datetime
        self.fname = fname
        super(MyCallback, self).__init__()
        with open(self.fname, "a") as f:
            f.write("Logger initialized on {}".format(datetime.datetime.today().isoformat()))
        
    def on_epoch_end(self, epoch, logs={}):
        content = "Epoch: {}, loss: {}\n".format(epoch, logs.get('loss'))
        with open(self.fname, "a") as f:
            f.write(content)

class StopOnConvergence(Callback):
    def __init__(self, max_repetitions=10):
        super().__init__()
        self.max_repetitions = max_repetitions

    def on_train_begin(self, logs=None):
        self.repetitions = 0
        self.last_loss = np.inf

    def on_epoch_end(self, batch, logs=None):
        logs = logs or {}
        loss = logs.get('val_loss')
        if loss is not None:
            if loss > self.last_loss:
                self.repetitions += 1
            else:
                self.last_loss = loss
                self.repetitions = 0
            if self.repetitions > self.max_repetitions:
                self.model.stop_training = True

            
logger = MyCallback("./training-bigger-log.txt")
NFEATS = 1


class AdamScale(Adam):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self._scale_decay = 0.999
        self.var = None 
    
    @keras.legacy.interfaces.legacy_get_updates_support
    def get_updates(self, loss, params):
        if self.var is None:
            self.var = K.variable([1.], name='czt_scale')
            self.weights.append(self.var)
        super().get_updates(loss, params)
        self.updates = [(self.var * x if x.shape.as_list() == [2] else x) for x in self.updates]
        self.updates.append((self.var, self._scale_decay * self.var))
        return self.updates

shift = np.exp(2 * np.pi * 1j / 512)

def train(model, trainX, trainy, trainXl, trainyl, epochs = 50):
    # important: batch_size=1 bugs Tensorflow
    optimizer = AdamScale(0.0003, clipnorm=1.)
    model.compile(loss={'ctc': lambda y_true, y_pred: y_pred}, optimizer=optimizer)
    return model.fit([trainX[~valid], trainy[~valid], trainXl[~valid], trainyl[~valid]], np.zeros(trainX[~valid].shape[0]), epochs = epochs,
                     batch_size = 8, callbacks=[mc, logger, StopOnConvergence(4)], validation_data=[[trainX[valid], trainy[valid], trainXl[valid], trainyl[valid]], np.zeros(trainX[valid].shape[0])])

def validate(predictions, valid_length, groundtruth, target_length):
    predictions = keras.backend.ctc_decode(predictions, valid_length, False, 1)
    predictions = predictions[0][0].eval(session=keras.backend.get_session())
    DERs = []
    for index in range(predictions.shape[0]):
        dist = float(editdistance.eval(
            [x for x in predictions[index, :] if x != -1],
            [x for x in groundtruth[index, :] if x != NPHONES]))
        DER = dist / target_length[index]
        DERs.append((DER, target_length[index]))
    return DERs

def try_else(exp, exp_else):
    try:
        return exp()
    except:
        return exp_else

if __name__=='__main__':
    data = X, Y
    NPHONES = Y.max()
    NFEATS = data[0].shape[2]
    X_lens = np.array([first(x) // (4 * 128) for x in X]).astype(np.int64)
    Y_lens = np.array([np.where(x == NPHONES)[0] for x in data[1]])
    Y_lens = np.array([x[0] if len(x) else 0 for x in Y_lens]).astype(np.int64)
    print(X.shape, Y.shape, X_lens.shape, Y_lens.shape)
    data = data[0][np.where(X_lens)], data[1][np.where(X_lens)], X_lens[np.where(Y_lens)].reshape(-1, 1), Y_lens[np.where(Y_lens)].reshape(-1, 1)
    print(list(map(lambda x:x.shape, data)))
    trn, predict = mk_model(data[1].shape[1])
    train(trn, *data, epochs=100) # at 300 it makes sensible predictions


(5504, 217600, 1) (5504, 307) (5504,) (5504,)
[(5504, 217600, 1), (5504, 307), (5504, 1), (5504, 1)]


/venv/lib/python3.5/site-packages/ipykernel_launcher.py:36: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(512, 512, use_bias=False, activation="relu", strides=128)`


() () (257, 512)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, None, 1)      0                                            
__________________________________________________________________________________________________
conv1d_3 (Conv1D)               (None, None, 512)    262144      input_3[0][0]                    
__________________________________________________________________________________________________
czt_2 (CZT)                     (None, None, 257)    4           conv1d_3[0][0]                   
__________________________________________________________________________________________________
lambda_5 (Lambda)               (None, None, 20)     0           czt_2[0][0]                      
____________________________________________________________________________________________

Train on 4948 samples, validate on 556 samples
Epoch 1/100
4948/4948 [==============================] - 4351s 879ms/step - loss: 565.1582 - val_loss: 468.0357
Epoch 2/100
4948/4948 [==============================] - 4325s 874ms/step - loss: 469.6810 - val_loss: 476.2239
Epoch 3/100
4948/4948 [==============================] - 4325s 874ms/step - loss: 443.7300 - val_loss: 455.4404
Epoch 4/100
4948/4948 [==============================] - 4323s 874ms/step - loss: 416.1790 - val_loss: 403.4757
Epoch 5/100
4948/4948 [==============================] - 4325s 874ms/step - loss: 402.8094 - val_loss: 397.0816
Epoch 6/100
4948/4948 [==============================] - 4325s 874ms/step - loss: 394.4494 - val_loss: 414.0718
Epoch 7/100
4948/4948 [==============================] - 4325s 874ms/step - loss: 388.1816 - val_loss: 384.8984
Epoch 8/100
4948/4948 [==============================] - 4325s 874ms/step - loss: 384.3081 - val_loss: 388.4656
Epoch 9/100
4948/4948 [==============================] - 

<tf.Tensor 'Tile_8:0' shape=(?, ?, 1) dtype=float32>

In [24]:
pr = predict.predict(X[:1])
pr = K.ctc_decode(pr, X_lens[:1])[0][0]
pr.eval(session=K.get_session())

array([[18,  4, 20, 13,  5, 26, 13,  5, 22, 17,  8, 35, 27, 22, 25,  8,
        29, 13,  9, 21,  9, 29, 10, 34,  8, 24,  8, 35, 27, 22, 35,  9,
        11, 32,  9, 18, 20, 13, 13, 20, 13,  9, 24,  6, 31, 20,  7, 10,
        11, 12, 22, 13,  5, 30,  8,  5, 20, 25, 18]])

In [13]:
(Y[:] == 37).mean()

0.28032499905310204

In [26]:
import editdistance
editdistance.eval(list(pr.eval(session=K.get_session())[0]), list(Y[0, :Y_lens[0]])) / Y_lens[0]

0.37142857142857144

In [27]:
pr = predict.predict(X[:160])
pr = K.ctc_decode(pr, X_lens[:160])[0][0]
hypos = [list(x) for x in pr.eval(session=K.get_session())]
lens = [x.index(-1) if -1 in x else len(x) for x in hypos]
hypos = [x[:lim] for x, lim in zip(hypos, lens)]
gts = [list(x) for x in Y[:160]]
lens = [x.index(37) if 37 in x else len(x) for x in gts]
gts = [x[:lim] for x, lim in zip(gts, lens)]
[editdistance.eval(gt, hypo) / len(gt) for gt, hypo in zip(gts, hypos)]

[0.37142857142857144,
 0.32558139534883723,
 0.4032258064516129,
 0.4375,
 0.4214876033057851,
 0.3829787234042553,
 0.2777777777777778,
 0.25555555555555554,
 0.35294117647058826,
 0.43478260869565216,
 0.43037974683544306,
 0.3472222222222222,
 0.35384615384615387,
 0.5441176470588235,
 0.48717948717948717,
 0.4117647058823529,
 0.5,
 0.5,
 0.3707865168539326,
 0.4457831325301205,
 0.32142857142857145,
 0.45348837209302323,
 0.3711340206185567,
 0.5196078431372549,
 0.3269230769230769,
 0.39603960396039606,
 0.36363636363636365,
 0.48484848484848486,
 0.3333333333333333,
 0.5050505050505051,
 0.4157303370786517,
 0.4305555555555556,
 0.345679012345679,
 0.417910447761194,
 0.375,
 0.38461538461538464,
 0.42168674698795183,
 0.4430379746835443,
 0.4186046511627907,
 0.42528735632183906,
 0.3950617283950617,
 0.44047619047619047,
 0.3614457831325301,
 0.39080459770114945,
 0.5714285714285714,
 0.45652173913043476,
 0.4507042253521127,
 0.3924050632911392,
 0.47297297297297297,
 0.43859

In [28]:
np.array(_).mean()

0.41013983375959473

In [29]:
K.zeros([2]).shape.as_list() == [2]

True

In [30]:
trn.optimizer.weights

[<tf.Variable 'AdamScale/iterations:0' shape=() dtype=int64_ref>,
 <tf.Variable 'training/AdamScale/Variable:0' shape=(2,) dtype=float32_ref>,
 <tf.Variable 'training/AdamScale/Variable_1:0' shape=(2,) dtype=float32_ref>,
 <tf.Variable 'training/AdamScale/Variable_2:0' shape=(5, 1, 1, 12) dtype=float32_ref>,
 <tf.Variable 'training/AdamScale/Variable_3:0' shape=(12,) dtype=float32_ref>,
 <tf.Variable 'training/AdamScale/Variable_4:0' shape=(12,) dtype=float32_ref>,
 <tf.Variable 'training/AdamScale/Variable_5:0' shape=(12,) dtype=float32_ref>,
 <tf.Variable 'training/AdamScale/Variable_6:0' shape=(5, 1, 12, 16) dtype=float32_ref>,
 <tf.Variable 'training/AdamScale/Variable_7:0' shape=(16,) dtype=float32_ref>,
 <tf.Variable 'training/AdamScale/Variable_8:0' shape=(16,) dtype=float32_ref>,
 <tf.Variable 'training/AdamScale/Variable_9:0' shape=(16,) dtype=float32_ref>,
 <tf.Variable 'training/AdamScale/Variable_10:0' shape=(5, 320, 512) dtype=float32_ref>,
 <tf.Variable 'training/AdamScal

In [31]:
K.variable([9]).eval(session=K.get_session())

array([9.], dtype=float32)

In [32]:
predict.save("models/czt-stabilized-coeff-columns.h5")

In [40]:
pr = predict.predict(X[:1])
pr = K.ctc_decode(pr, X_lens[:1])[0][0]
letters = pr.eval(session=K.get_session())
[lang[x] for x in letters[0]]

['sil',
 'Z',
 'a',
 's',
 't',
 'u',
 's',
 't',
 'e',
 'ni',
 'i',
 'p',
 'S',
 'e',
 'k',
 'i',
 'x',
 's',
 'I',
 'v',
 'I',
 'x',
 'o',
 'dzi',
 'i',
 'm',
 'i',
 'p',
 'S',
 'e',
 'p',
 'I',
 'si',
 'n',
 'I',
 'sil',
 'a',
 's',
 's',
 'a',
 's',
 'I',
 'm',
 'f',
 'tsi',
 'a',
 'ts',
 'o',
 'si',
 'j',
 'e',
 's',
 't',
 'l',
 'i',
 't',
 'a',
 'k',
 'sil']

In [44]:
[lang[x] for x in Y[0][:Y_lens[0]]]

['sil',
 'tsi',
 'a',
 's',
 't',
 'o',
 'sil',
 's',
 'p',
 'a',
 'ni',
 'i',
 'p',
 'S',
 'e',
 'sil',
 'p',
 'i',
 's',
 'u',
 'v',
 'I',
 'x',
 'o',
 'dzi',
 'i',
 'm',
 'sil',
 'i',
 'p',
 'sil',
 'S',
 'e',
 'p',
 'I',
 'sil',
 'S',
 'n',
 'e',
 'sil',
 'a',
 'sil',
 's',
 's',
 'o',
 's',
 'e',
 'sil',
 'm',
 'f',
 'tsi',
 'o',
 'n',
 'S',
 'sil',
 'ts',
 'o',
 'si',
 'j',
 'e',
 'sil',
 's',
 't',
 'ni',
 'e',
 't',
 'sil',
 'a',
 'k',
 'sil']